In [ ]:
!pip install pyyaml h5py
!pip install transformers
!pip install scipy
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel
import tensorflow as tf
from transformers import RobertaTokenizer, RobertaModel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.1 MB/s 
     |████████████████████████████████| 163 kB 69.1 MB/s 
     |████████████████████████████████| 7.6 MB 45.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open('PennsylvaniaMidterms11Aug11Oct2').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
import pandas as pd
df=pd.DataFrame.from_records(rows)




In [ ]:
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

In [ ]:
# import pandas as pd
# import numpy as np
# df = pd.read_excel('/content/drive/MyDrive/NLP_Project/labelled_test.xlsx')

In [ ]:
df['SentimentScore']= pd.to_numeric(df['SentimentScore'])
df['Subjectivity']= pd.to_numeric(df['Subjectivity'])
df['Sarcasm']= pd.to_numeric(df['Sarcasm'])
df['Faction']= pd.to_numeric(df['Faction'])
final_df=df.loc[df['SentimentScore'].isin([0,1,2])]

# set seed

In [ ]:
SEED = 42

import os
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

import random 
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

# data preprocessing

In [ ]:
import re
import nltk

In [ ]:

def clean_content(content):
  content=re.sub('#','',content)
  content = re.sub(r'@[^(JohnFetterman)][^(DrOz)]\w+', '', content)
  content = re.sub('@', '', content)
  content = re.sub(r'\\u\w+', '', content)
  content = re.sub(r"http\S+", "", content)
  content = re.sub("\n", "", content)
  # content = re.sub(r'[^\w\s\.]', '', content)
  # content=' '.join(s for s in content.split() if not any(c.isdigit() for c in s))
  # content=content.lower()
  return content


In [ ]:
final_df['cleaned_tweets']=final_df['Content'].apply(clean_content)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def nltk2wn_tag(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:                    
    return None
def lemmatize_sentence(content,lemmatizer=lemmatizer):
  nltk_tagged = nltk.pos_tag(nltk.word_tokenize(content))    
  wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
  res_words = []
  for word, tag in wn_tagged:
    if tag is None:             
      res_words.append(word.lower())
    else:
      res_words.append(lemmatizer.lemmatize(word.lower(), tag))
  content=" ".join(res_words)
  content = re.sub(r'[^\w\s]', '', content)
  content=' '.join(s for s in content.split() if not any(c.isdigit() for c in s))
  return content

In [ ]:
final_df['lemmatized_tweets']=final_df['cleaned_tweets'].apply(lemmatize_sentence)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
final_df

,Date,Content,,,,,,,,,,,,,SentimentScore,Subjectivity,Sarcasm,Faction,cleaned_tweets,lemmatized_tweets
1,2022-10-08 15:43:14+00:00,VOTE FOR @JohnFetterman .\nRepublican @DrOz w...,1578773016908951554,Richard79300111,0,0,1,0,1578773016908951554,,https://twitter.com/JohnFetterman/status/15787...,,,"[User(username='JohnFetterman', id=3622368202,...",0.0,1.0,1.0,1.0,VOTE FOR JohnFetterman .Republican DrOz will ...,vote for johnfetterman republican droz will no...
2,2022-09-23 21:35:23+00:00,@the_vello Why does he always look like a SLOB...,1573425822970568709,sgtmolly06,0,0,6,0,1573419293273907211,,,1.57E+18,https://twitter.com/the_vello,"[User(username='the_vello', id=93330058, displ...",0.0,1.0,0.0,1.0,the_vello Why does he always look like a SLOB?...,the_vello why do he always look like a slob he...
3,2022-09-23 18:46:26+00:00,John Fetterman is a POS! Just a trust found ki...,1573383305566846976,aizenlorde,0,1,1,0,1573383305566846976,,,,,,0.0,1.0,1.0,1.0,John Fetterman is a POS! Just a trust found ki...,john fetterman be a po just a trust find kid w...
4,2022-09-23 16:04:47+00:00,"@DrOz I would love to see a sit-down, face-to-...",1573342622877626369,darthbanish_EP2,0,0,1,0,1573342057321619456,,,1.57E+18,https://twitter.com/DrOz,"[User(username='DrOz', id=38531995, displaynam...",0.0,1.0,1.0,1.0,"DrOz I would love to see a sit-down, face-to-f...",droz i would love to see a sitdown facetoface ...
5,2022-09-23 13:37:39+00:00,Baphomet is non binary and babies were slain f...,1573305596190904321,Nanaof313,0,0,3,0,1573305596190904321,,,,,,0.0,1.0,0.0,1.0,Baphomet is non binary and babies were slain f...,baphomet be non binary and baby be slay for mo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,2022-08-31 17:45:43+00:00,#JohnFetterman stroke recovery is absolutely f...,1565033103860572162,MarkDavis,4,0,22,1,1565033103860572162,,https://twitter.com/TomBevanRCP/status/1565030...,,,,2.0,0.0,NaN,2.0,JohnFetterman stroke recovery is absolutely fa...,johnfetterman stroke recovery be absolutely fa...
1517,2022-08-31 16:04:29+00:00,@NRSC_Rapid @JohnFetterman @DrOz Continuing hi...,1565007625900752898,SnowyEvergreen,0,0,0,0,1564325011007176708,,,1.56E+18,https://twitter.com/NRSC_Rapid,"[User(username='NRSC_Rapid', id=13592192524615...",2.0,0.0,NaN,1.0,JohnFetterman DrOz Continuing his campaign on...,johnfetterman droz continue his campaign only ...
1525,2022-08-31 04:45:49+00:00,@sahilkapur @JohnFetterman @DrOz I think #John...,1564836833397067776,StopEndWarsNOW,0,0,0,0,1564758762115719169,,,1.56E+18,https://twitter.com/sahilkapur,"[User(username='sahilkapur', id=19847765, disp...",2.0,0.0,NaN,1.0,JohnFetterman DrOz I think JohnFetterman shou...,johnfetterman droz i think johnfetterman shoul...
1548,2022-08-30 05:35:45+00:00,"He's a real winner, isn't he? #JohnFetterman i...",1564487012811476995,rlburson,1,0,1,0,1564487012811476995,,https://twitter.com/RNCResearch/status/1564418...,,,,0.0,1.0,1.0,1.0,"He's a real winner, isn't he? JohnFetterman is...",he s a real winner be nt he johnfetterman be b...


# metric

In [ ]:
#see result
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
def get_result(target,pred):
  '''returns accuracy, f1, precision, recall'''
  return accuracy_score(target,pred),f1_score(target,pred,average='weighted'),precision_score(target,pred),recall_score(target,pred)


# train test split and subjectivity detection

In [ ]:
tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")
def tokenize_roberta(data,max_len=128,tokenizer_roberta=tokenizer_roberta) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
def roberta_model(bert_model, max_len=128):
    
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.BinaryAccuracy()

    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(128, activation='relu')(output)
    output = tf.keras.layers.Dense(1)(output)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(opt, loss=loss, metrics=accuracy)
    return model

In [ ]:
#five fold X validation
from sklearn.model_selection import KFold,StratifiedKFold

skf=KFold(n_splits=5, shuffle=True, random_state=SEED)
accuracy=[]
f1=[]
precision=[]
recall=[]
roberta = TFRobertaModel.from_pretrained('roberta-base')
best_model = roberta_model(roberta)
best_a=0
for train_index, test_index in skf.split(final_df['cleaned_tweets'].values,final_df['Subjectivity'].values):
  X_train, X_test = final_df['cleaned_tweets'].values[train_index], final_df['cleaned_tweets'].values[test_index]
  y_train, y_test = final_df['Subjectivity'].values[train_index], final_df['Subjectivity'].values[test_index]
  roberta = TFRobertaModel.from_pretrained('roberta-base')
  r_model = roberta_model(roberta)
  train_input_ids, train_attention_masks = tokenize_roberta(X_train, 128)
  test_input_ids, test_attention_masks = tokenize_roberta(X_test, 128)
  history = r_model.fit([train_input_ids,train_attention_masks], y_train, epochs=4, batch_size=32)
  result_roberta = r_model.predict([test_input_ids,test_attention_masks])
  result_roberta=tf.math.sigmoid(result_roberta)
  result_roberta=tf.reshape(result_roberta,(result_roberta.shape[0],)).numpy()
  y_pred=[]
  for i in result_roberta:
    if i<=0.5:
      y_pred.append(0)

    else:
      y_pred.append(1)
  a,b,c,d=get_result(y_test,y_pred)
  if a > best_a:
    best_model=r_model
    best_a=a
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)

best_model.save_weights('/content/drive/MyDrive/model/subjectivity.h5')

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel f

Epoch 1/4
27/27 [==============================] - 50s 727ms/step - loss: 0.6454 - binary_accuracy: 0.4571
Epoch 2/4
27/27 [==============================] - 20s 726ms/step - loss: 0.3341 - binary_accuracy: 0.8929
Epoch 3/4
27/27 [==============================] - 20s 744ms/step - loss: 0.2913 - binary_accuracy: 0.8988
Epoch 4/4
7/7 [==============================] - 4s 242ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
27/27 [==============================] - 36s 749ms/step - loss: 0.5578 - binary_accuracy: 0.5600
Epoch 2/4
27/27 [==============================] - 21s 761ms/step - loss: 0.3549 - binary_accuracy: 0.8811
Epoch 3/4
27/27 [==============================] - 21s 769ms/step - loss: 0.3010 - binary_accuracy: 0.8847
Epoch 4/4
7/7 [==============================] - 4s 238ms/step


...

Epoch 1/4
27/27 [==============================] - 36s 760ms/step - loss: 0.4486 - binary_accuracy: 0.7111
Epoch 2/4
27/27 [==============================] - 21s 769ms/step - loss: 0.3362 - binary_accuracy: 0.8918
Epoch 3/4
27/27 [==============================] - 21s 765ms/step - loss: 0.3032 - binary_accuracy: 0.8918
Epoch 4/4
7/7 [==============================] - 4s 238ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
27/27 [==============================] - 36s 763ms/step - loss: 0.3913 - binary_accuracy: 0.8930
Epoch 2/4
27/27 [==============================] - 21s 775ms/step - loss: 0.3356 - binary_accuracy: 0.8930
Epoch 3/4
27/27 [==============================] - 21s 760ms/step - loss: 0.2931 - binary_accuracy: 0.8930
Epoch 4/4
7/7 [==============================] - 4s 238ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
27/27 [==============================] - 37s 767ms/step - loss: 0.4648 - binary_accuracy: 0.6968
Epoch 2/4
27/27 [==============================] - 21s 776ms/step - loss: 0.3378 - binary_accuracy: 0.8894
Epoch 3/4
27/27 [==============================] - 21s 763ms/step - loss: 0.3149 - binary_accuracy: 0.8894
Epoch 4/4
7/7 [==============================] - 4s 240ms/step


In [ ]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.8924937937260212, f1: 0.8635544260166454, precision: 0.9138093631514683, recall: 0.9734745054332684


# textblob subjectivity

In [ ]:
from textblob import TextBlob
def getSubjectivity(review):
    return TextBlob(review).sentiment.subjectivity

In [ ]:
textblob_sub=final_df['cleaned_tweets'].apply(getSubjectivity)

In [ ]:
y_pred=[]
for i in textblob_sub:
  if i<=0.5:
    y_pred.append(0)

  else:
    y_pred.append(1)

In [ ]:
print(get_result(final_df['Subjectivity'].values,y_pred))

(0.3939105613701237, 0.4729389734252783, 0.9257142857142857, 0.34652406417112297)


# get raw sentiment

In [ ]:
sentiment=final_df[final_df['SentimentScore']<2]
sentiment=sentiment[sentiment['Faction']<2]
sentiment=sentiment.dropna(subset=['Faction'])

  

In [ ]:
support=sentiment['SentimentScore'].values
faction=sentiment['Faction'].values
raw_sent=[]
for i in range(len(faction)):
  if faction[i]==2:
    raw_sent.append(support[i])
  elif faction[i]!=support[i]:
    if support[i]==0:
      raw_sent.append(1)
    else:
      raw_sent.append(0)
  else:
    raw_sent.append(support[i])

# nltk

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
def nltk_sent(content):
  sia = SentimentIntensityAnalyzer()
  if sia.polarity_scores(content)['pos'] > sia.polarity_scores(content)['neg']:
    s=1
  else:
    s=0
  return s

In [ ]:
nltk_score=sentiment['cleaned_tweets'].apply(nltk_sent)
print(get_result(raw_sent,nltk_score))

(0.4737991266375546, 0.47581874007643227, 0.7350993377483444, 0.3557692307692308)


In [ ]:
nltk_score=sentiment['lemmatized_tweets'].apply(nltk_sent)
print(get_result(raw_sent,nltk_score))

(0.47925764192139736, 0.4853893450161035, 0.7261538461538461, 0.3782051282051282)


#textblob

In [ ]:
from textblob import TextBlob
def text_score(content):
  if TextBlob(content).sentiment[1]<=0.5:
    s=0
  else:
    s=1
  return s

In [ ]:
txt_score=sentiment['cleaned_tweets'].apply(text_score)
print(get_result(raw_sent,txt_score))

(0.42248908296943233, 0.4277756197127178, 0.6498422712933754, 0.3301282051282051)


In [ ]:
txt_score=sentiment['lemmatized_tweets'].apply(text_score)
print(get_result(raw_sent,txt_score))

(0.45087336244541487, 0.4617403947682731, 0.6713881019830028, 0.3798076923076923)


#roberta on raw sentiment

In [ ]:
from torch.nn.modules.module import register_module_full_backward_hook
accuracy=[]
f1=[]
precision=[]
recall=[]
roberta = TFRobertaModel.from_pretrained('roberta-base')
best_model = roberta_model(roberta)
best_a=0
for train_index, test_index in skf.split(sentiment['cleaned_tweets'].values,np.array(raw_sent)):
  X_train, X_test = sentiment['cleaned_tweets'].values[train_index], sentiment['cleaned_tweets'].values[test_index]
  y_train, y_test = np.array(raw_sent)[train_index], np.array(raw_sent)[test_index]
  roberta = TFRobertaModel.from_pretrained('roberta-base')
  r_model = roberta_model(roberta)
  train_input_ids, train_attention_masks = tokenize_roberta(X_train, 128)
  test_input_ids, test_attention_masks = tokenize_roberta(X_test, 128)
  history = r_model.fit([train_input_ids,train_attention_masks], y_train, epochs=5, batch_size=32)
  result_roberta = r_model.predict([test_input_ids,test_attention_masks])
  result_roberta=tf.math.sigmoid(result_roberta)
  result_roberta=tf.reshape(result_roberta,(result_roberta.shape[0],)).numpy()
  y_pred=[]
  for i in result_roberta:
    if i<=0.5:
      y_pred.append(0)

    else:
      y_pred.append(1)
  a,b,c,d=get_result(y_test,y_pred)
  if a > best_a:
    best_model=r_model
    best_a=a
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel f

Epoch 1/5
23/23 [==============================] - 35s 779ms/step - loss: 0.6480 - binary_accuracy: 0.5301
Epoch 2/5
23/23 [==============================] - 18s 794ms/step - loss: 0.5987 - binary_accuracy: 0.7049
Epoch 3/5
23/23 [==============================] - 18s 785ms/step - loss: 0.4258 - binary_accuracy: 0.8224
Epoch 4/5
23/23 [==============================] - 18s 775ms/step - loss: 0.2987 - binary_accuracy: 0.8798
Epoch 5/5
6/6 [==============================] - 4s 245ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/5
23/23 [==============================] - 33s 783ms/step - loss: 0.6496 - binary_accuracy: 0.5211
Epoch 2/5
23/23 [==============================] - 18s 793ms/step - loss: 0.5914 - binary_accuracy: 0.7258
Epoch 3/5
23/23 [==============================] - 18s 782ms/step - loss: 0.4021 - binary_accuracy: 0.8240
Epoch 4/5
23/23 [==============================] - 18s 775ms/step - loss: 0.2778 - binary_accuracy: 0.8840
Epoch 5/5
6/6 [==============================] - 4s 244ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/5
23/23 [==============================] - 33s 786ms/step - loss: 0.6560 - binary_accuracy: 0.4625
Epoch 2/5
23/23 [==============================] - 18s 796ms/step - loss: 0.6172 - binary_accuracy: 0.6862
Epoch 3/5
23/23 [==============================] - 18s 782ms/step - loss: 0.5097 - binary_accuracy: 0.7804
Epoch 4/5
23/23 [==============================] - 18s 776ms/step - loss: 0.3185 - binary_accuracy: 0.8649
Epoch 5/5
6/6 [==============================] - 4s 243ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/5
23/23 [==============================] - 34s 782ms/step - loss: 0.6164 - binary_accuracy: 0.6494
Epoch 2/5
23/23 [==============================] - 18s 795ms/step - loss: 0.5892 - binary_accuracy: 0.7394
Epoch 3/5
23/23 [==============================] - 18s 782ms/step - loss: 0.4442 - binary_accuracy: 0.8186
Epoch 4/5
23/23 [==============================] - 18s 775ms/step - loss: 0.3019 - binary_accuracy: 0.8799
Epoch 5/5
6/6 [==============================] - 4s 247ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/5
23/23 [==============================] - 34s 780ms/step - loss: 0.6216 - binary_accuracy: 0.5457
Epoch 2/5
23/23 [==============================] - 18s 796ms/step - loss: 0.5791 - binary_accuracy: 0.7462
Epoch 3/5
23/23 [==============================] - 18s 785ms/step - loss: 0.3599 - binary_accuracy: 0.8554
Epoch 4/5
23/23 [==============================] - 18s 776ms/step - loss: 0.2708 - binary_accuracy: 0.8990
Epoch 5/5
6/6 [==============================] - 4s 243ms/step


In [ ]:
best_model.save_weights('/content/drive/MyDrive/sentiment_model/sentiment.h5')

In [ ]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.863512710857686, f1: 0.8595363800645522, precision: 0.8694842177908539, recall: 0.9411400225083423


# svm using tfidf on raw sentiment

In [ ]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

accuracy=[]
f1=[]
precision=[]
recall=[]
best_model=svm.SVC()
best_a=0
for train_index, test_index in skf.split(sentiment['cleaned_tweets'].values,np.array(raw_sent)):
  X_train, X_test = sentiment['cleaned_tweets'].values[train_index], sentiment['cleaned_tweets'].values[test_index]
  y_train, y_test = np.array(raw_sent)[train_index], np.array(raw_sent)[test_index]
  train_vectors = vectorizer.fit_transform(X_train)
  test_vectors = vectorizer.transform(X_test)
  model = svm.SVC()
  y_pred = model.fit(train_vectors, y_train).predict(test_vectors)
  a,b,c,d=get_result(y_test,y_pred)
  if a > best_a:
    best_model=model
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)


In [ ]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.7936029935851746, f1: 0.7742883171238215, precision: 0.7898194732239047, recall: 0.9507284711176863


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

accuracy=[]
f1=[]
precision=[]
recall=[]
for train_index, test_index in skf.split(sentiment['lemmatized_tweets'].values,np.array(raw_sent)):
  X_train, X_test = sentiment['lemmatized_tweets'].values[train_index], sentiment['cleaned_tweets'].values[test_index]
  y_train, y_test = np.array(raw_sent)[train_index], np.array(raw_sent)[test_index]
  train_vectors = vectorizer.fit_transform(X_train)
  test_vectors = vectorizer.transform(X_test)
  model = svm.SVC()
  y_pred = model.fit(train_vectors, y_train).predict(test_vectors)
  a,b,c,d=get_result(y_test,y_pred)
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)

In [ ]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.7805238774055595, f1: 0.7598698817675446, precision: 0.7811246735760168, recall: 0.9432684955612254


# another naive bayes on raw sentiment

In [ ]:
class Vocabulary:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {}
    self.num_words = 0

  def add_word(self, word):
    if word not in self.word2index:
      # First entry of word into vocabulary
      self.word2index[word] = self.num_words
      self.word2count[word] = 1
      self.index2word[self.num_words] = word
      self.num_words += 1
    else:
      # Word exists; increase word count
      self.word2count[word] += 1

In [ ]:
from nltk.tokenize import wordpunct_tokenize

vocab=Vocabulary('tweet')
for i in sentiment['lemmatized_tweets']:
  result=wordpunct_tokenize(i)
  for j in result:
    vocab.add_word(j)
def vectorize(input,vocab=vocab):
  master_list=[]
  for i in input:
    result=wordpunct_tokenize(i)
    tweet=[0]*len(vocab.word2index)
    for j in result:
      for k in vocab.word2index:
        if j==k:
          tweet[vocab.word2index[j]]=1
    master_list.append(tweet)
  return master_list

from sklearn.naive_bayes import MultinomialNB

accuracy=[]
f1=[]
precision=[]
recall=[]
best_model=MultinomialNB()
best_a=0
for train_index, test_index in skf.split(sentiment['lemmatized_tweets'].values,np.array(raw_sent)):
  X_train, X_test = sentiment['lemmatized_tweets'].values[train_index], sentiment['cleaned_tweets'].values[test_index]
  y_train, y_test = np.array(raw_sent)[train_index], np.array(raw_sent)[test_index]
  train_vectors = vectorize(X_train)
  test_vectors = vectorize(X_test)
  model = MultinomialNB()
  y_pred = model.fit(vectorize(X_train), y_train).predict(vectorize(X_test))
  if a > best_a:
    best_a=a
    best_model=model
  a,b,c,d=get_result(y_test,y_pred)
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)

In [ ]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.7390413399857448, f1: 0.7043353372445881, precision: 0.7455220447137336, recall: 0.9377429445889724


# roberta on support

In [ ]:
support_df=final_df[final_df['SentimentScore']<2]

In [ ]:
accuracy=[]
f1=[]
precision=[]
recall=[]
roberta = TFRobertaModel.from_pretrained('roberta-base')
for train_index, test_index in skf.split(support_df['cleaned_tweets'].values,support_df['SentimentScore'].values):
  X_train, X_test = support_df['cleaned_tweets'].values[train_index], support_df['cleaned_tweets'].values[test_index]
  y_train, y_test = support_df['SentimentScore'].values[train_index], support_df['SentimentScore'].values[test_index]
  roberta = TFRobertaModel.from_pretrained('roberta-base')
  r_model = roberta_model(roberta)
  train_input_ids, train_attention_masks = tokenize_roberta(X_train, 128)
  test_input_ids, test_attention_masks = tokenize_roberta(X_test, 128)
  history = r_model.fit([train_input_ids,train_attention_masks], y_train, epochs=4, batch_size=32)
  result_roberta = r_model.predict([test_input_ids,test_attention_masks])
  result_roberta=tf.math.sigmoid(result_roberta)
  result_roberta=tf.reshape(result_roberta,(result_roberta.shape[0],)).numpy()
  y_pred=[]
  for i in result_roberta:
    if i<=0.5:
      y_pred.append(0)

    else:
      y_pred.append(1)
  a,b,c,d=get_result(y_test,y_pred)
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel f

Epoch 1/4
24/24 [==============================] - 33s 765ms/step - loss: 0.6829 - binary_accuracy: 0.4439
Epoch 2/4
24/24 [==============================] - 19s 778ms/step - loss: 0.6466 - binary_accuracy: 0.5401
Epoch 3/4
24/24 [==============================] - 18s 765ms/step - loss: 0.4797 - binary_accuracy: 0.7861
Epoch 4/4
6/6 [==============================] - 4s 249ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
24/24 [==============================] - 34s 762ms/step - loss: 0.6799 - binary_accuracy: 0.4425
Epoch 2/4
24/24 [==============================] - 19s 777ms/step - loss: 0.6161 - binary_accuracy: 0.6818
Epoch 3/4
24/24 [==============================] - 18s 768ms/step - loss: 0.4354 - binary_accuracy: 0.7995
Epoch 4/4
6/6 [==============================] - 4s 249ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
24/24 [==============================] - 34s 760ms/step - loss: 0.6817 - binary_accuracy: 0.4666
Epoch 2/4
24/24 [==============================] - 19s 775ms/step - loss: 0.6152 - binary_accuracy: 0.6684
Epoch 3/4
24/24 [==============================] - 19s 771ms/step - loss: 0.3797 - binary_accuracy: 0.8329
Epoch 4/4
6/6 [==============================] - 4s 248ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
24/24 [==============================] - 34s 760ms/step - loss: 0.6976 - binary_accuracy: 0.4826
Epoch 2/4
24/24 [==============================] - 19s 776ms/step - loss: 0.6126 - binary_accuracy: 0.6725
Epoch 3/4
24/24 [==============================] - 18s 768ms/step - loss: 0.3866 - binary_accuracy: 0.8262
Epoch 4/4
6/6 [==============================] - 4s 248ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
24/24 [==============================] - 34s 760ms/step - loss: 0.6857 - binary_accuracy: 0.4519
Epoch 2/4
24/24 [==============================] - 19s 778ms/step - loss: 0.6531 - binary_accuracy: 0.6070
Epoch 3/4
24/24 [==============================] - 18s 768ms/step - loss: 0.4872 - binary_accuracy: 0.7741
Epoch 4/4
6/6 [==============================] - 5s 252ms/step


In [ ]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.8481283422459892, f1: 0.8440870269988702, precision: 0.8266458135228467, recall: 0.9322577412560127


# naive bayes on support

In [ ]:
vocab=Vocabulary('tweet')
for i in support_df['lemmatized_tweets']:
  result=wordpunct_tokenize(i)
  for j in result:
    vocab.add_word(j)
def vectorize(input,vocab=vocab):
  master_list=[]
  for i in input:
    result=wordpunct_tokenize(i)
    tweet=[0]*len(vocab.word2index)
    for j in result:
      for k in vocab.word2index:
        if j==k:
          tweet[vocab.word2index[j]]=1
    master_list.append(tweet)
  return master_list

from sklearn.naive_bayes import MultinomialNB

accuracy=[]
f1=[]
precision=[]
recall=[]
for train_index, test_index in skf.split(support_df['lemmatized_tweets'].values,support_df['SentimentScore'].values):
  X_train, X_test = support_df['lemmatized_tweets'].values[train_index], support_df['cleaned_tweets'].values[test_index]
  y_train, y_test = support_df['SentimentScore'].values[train_index], support_df['SentimentScore'].values[test_index]
  train_vectors = vectorize(X_train)
  test_vectors = vectorize(X_test)
  model = MultinomialNB()
  y_pred = model.fit(vectorize(X_train), y_train).predict(vectorize(X_test))
  a,b,c,d=get_result(y_test,y_pred)
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)

In [ ]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.722994652406417, f1: 0.7249684008325369, precision: 0.7957523446795765, recall: 0.6887192909425766


# svm on support

In [ ]:
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

accuracy=[]
f1=[]
precision=[]
recall=[]
for train_index, test_index in skf.split(support_df['lemmatized_tweets'].values,support_df['SentimentScore'].values):
  X_train, X_test = support_df['lemmatized_tweets'].values[train_index], support_df['lemmatized_tweets'].values[test_index]
  y_train, y_test = support_df['SentimentScore'].values[train_index], support_df['SentimentScore'].values[test_index]
  train_vectors = vectorizer.fit_transform(X_train)
  test_vectors = vectorizer.transform(X_test)
  model = svm.SVC()
  y_pred = model.fit(train_vectors, y_train).predict(test_vectors)
  a,b,c,d=get_result(y_test,y_pred)
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)

In [ ]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.8513368983957219, f1: 0.8506125382496192, precision: 0.8436342178750277, recall: 0.9048092478431446


# roberta on joint and aspect based

In [ ]:
def joint_model(bert_model, max_len=128):
    
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = {
	"Faction": tf.keras.losses.BinaryCrossentropy(from_logits=True),
	"RawSentiment": tf.keras.losses.BinaryCrossentropy(from_logits=True)}
    lossWeights = {"Faction": 1.0, "RawSentiment": 1.0}
    accuracy = tf.keras.metrics.BinaryAccuracy()

    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(128, activation='relu')(output)
    faction = tf.keras.layers.Dense(1,name='Faction')(output)
    sentiment = tf.keras.layers.Dense(1,name='RawSentiment')(output)
    output=[faction,sentiment]
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(opt, loss=loss, metrics=accuracy, loss_weights=lossWeights)
    return model

In [ ]:
joint_df=sentiment[['cleaned_tweets','Faction']]
joint_df['RawSentiment']=raw_sent

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
accuracy=[]
f1=[]
precision=[]
recall=[]

accuracy_s=[]
f1_s=[]
precision_s=[]
recall_s=[]
roberta = TFRobertaModel.from_pretrained('roberta-base')
best_model=joint_model(roberta)
best_a=0
for train_index, test_index in skf.split(joint_df['cleaned_tweets'].values,joint_df['Faction'].values):
  X_train, X_test = joint_df['cleaned_tweets'].values[train_index], joint_df['cleaned_tweets'].values[test_index]
  y_train, y_test = {'Faction':joint_df['Faction'].values[train_index], 'RawSentiment': joint_df['RawSentiment'].values[train_index]},{'Faction':joint_df['Faction'].values[test_index],'RawSentiment':joint_df['RawSentiment'].values[test_index]}
  roberta = TFRobertaModel.from_pretrained('roberta-base')
  r_model = joint_model(roberta)
  train_input_ids, train_attention_masks = tokenize_roberta(X_train, 128)
  test_input_ids, test_attention_masks = tokenize_roberta(X_test, 128)
  history = r_model.fit([train_input_ids,train_attention_masks], y_train, epochs=4, batch_size=32)
  result= r_model.predict([test_input_ids,test_attention_masks])
  f,s=result[0],result[1]
  f,s=tf.math.sigmoid(f),tf.math.sigmoid(s)
  f,s=tf.reshape(f,(f.shape[0],)).numpy(),tf.reshape(s,(s.shape[0],)).numpy()

  y_pred=[]
  for i in f:
    if i<=0.5:
      y_pred.append(0)

    else:
      y_pred.append(1)
  a,b,c,d=get_result(y_test['Faction'],y_pred)
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)
  if a>best_a:
    best_a=a
    best_model=r_model

  y_pred=[]
  for i in s:
    if i<=0.5:
      y_pred.append(0)

    else:
      y_pred.append(1)
  a,b,c,d=get_result(y_test['RawSentiment'],y_pred)
  accuracy_s.append(a)
  f1_s.append(b)
  precision_s.append(c)
  recall_s.append(d)
  

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel f

Epoch 1/4
23/23 [==============================] - 35s 777ms/step - loss: 1.3357 - Faction_loss: 0.6764 - RawSentiment_loss: 0.6593 - Faction_binary_accuracy: 0.4139 - RawSentiment_binary_accuracy: 0.4344
Epoch 2/4
23/23 [==============================] - 18s 795ms/step - loss: 1.2387 - Faction_loss: 0.6219 - RawSentiment_loss: 0.6167 - Faction_binary_accuracy: 0.7172 - RawSentiment_binary_accuracy: 0.7117
Epoch 3/4
23/23 [==============================] - 18s 784ms/step - loss: 0.9261 - Faction_loss: 0.4710 - RawSentiment_loss: 0.4552 - Faction_binary_accuracy: 0.8060 - RawSentiment_binary_accuracy: 0.8060
Epoch 4/4
6/6 [==============================] - 6s 247ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
23/23 [==============================] - 35s 786ms/step - loss: 1.2683 - Faction_loss: 0.6306 - RawSentiment_loss: 0.6378 - Faction_binary_accuracy: 0.5689 - RawSentiment_binary_accuracy: 0.5225
Epoch 2/4
23/23 [==============================] - 18s 795ms/step - loss: 1.0974 - Faction_loss: 0.5480 - RawSentiment_loss: 0.5494 - Faction_binary_accuracy: 0.7640 - RawSentiment_binary_accuracy: 0.7681
Epoch 3/4
23/23 [==============================] - 18s 787ms/step - loss: 0.7060 - Faction_loss: 0.3547 - RawSentiment_loss: 0.3513 - Faction_binary_accuracy: 0.8472 - RawSentiment_binary_accuracy: 0.8458
Epoch 4/4
6/6 [==============================] - 4s 241ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
23/23 [==============================] - 33s 778ms/step - loss: 1.3248 - Faction_loss: 0.6568 - RawSentiment_loss: 0.6680 - Faction_binary_accuracy: 0.4529 - RawSentiment_binary_accuracy: 0.4475
Epoch 2/4
23/23 [==============================] - 18s 796ms/step - loss: 1.1353 - Faction_loss: 0.5627 - RawSentiment_loss: 0.5725 - Faction_binary_accuracy: 0.7776 - RawSentiment_binary_accuracy: 0.7572
Epoch 3/4
23/23 [==============================] - 18s 786ms/step - loss: 0.8117 - Faction_loss: 0.4078 - RawSentiment_loss: 0.4039 - Faction_binary_accuracy: 0.8145 - RawSentiment_binary_accuracy: 0.8226
Epoch 4/4
6/6 [==============================] - 5s 244ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
23/23 [==============================] - 33s 777ms/step - loss: 1.2481 - Faction_loss: 0.6266 - RawSentiment_loss: 0.6215 - Faction_binary_accuracy: 0.5307 - RawSentiment_binary_accuracy: 0.5839
Epoch 2/4
23/23 [==============================] - 18s 796ms/step - loss: 1.2267 - Faction_loss: 0.6155 - RawSentiment_loss: 0.6112 - Faction_binary_accuracy: 0.6999 - RawSentiment_binary_accuracy: 0.6999
Epoch 3/4
23/23 [==============================] - 18s 786ms/step - loss: 1.0519 - Faction_loss: 0.5317 - RawSentiment_loss: 0.5202 - Faction_binary_accuracy: 0.7667 - RawSentiment_binary_accuracy: 0.7503
Epoch 4/4
6/6 [==============================] - 4s 243ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
23/23 [==============================] - 34s 775ms/step - loss: 1.2780 - Faction_loss: 0.6413 - RawSentiment_loss: 0.6366 - Faction_binary_accuracy: 0.4775 - RawSentiment_binary_accuracy: 0.4707
Epoch 2/4
23/23 [==============================] - 18s 798ms/step - loss: 1.2487 - Faction_loss: 0.6257 - RawSentiment_loss: 0.6230 - Faction_binary_accuracy: 0.6712 - RawSentiment_binary_accuracy: 0.7108
Epoch 3/4
23/23 [==============================] - 18s 788ms/step - loss: 0.9379 - Faction_loss: 0.4682 - RawSentiment_loss: 0.4697 - Faction_binary_accuracy: 0.7694 - RawSentiment_binary_accuracy: 0.7844
Epoch 4/4
6/6 [==============================] - 4s 244ms/step


In [ ]:
best_model.save_weights('/content/drive/MyDrive/model/joint.h5')

faction

In [ ]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.847119268234735, f1: 0.8464998405934828, precision: 0.8844184440697365, recall: 0.8959919039368245


sentiment

In [ ]:
print(f'accuracy: {sum(accuracy_s)/len(accuracy_s)}, f1: {sum(f1_s)/len(f1_s)}, precision: {sum(precision_s)/len(precision_s)}, recall: {sum(recall_s)/len(recall_s)}')

accuracy: 0.8515027322404372, f1: 0.8509212905041963, precision: 0.8857284044273754, recall: 0.9007328561404633


In [ ]:
test_input_ids, test_attention_masks = tokenize_roberta(['John Fetterman is awesome','DrOz is boring'], 128)
result = r_model.predict([test_input_ids,test_attention_masks])
f,s=result[0],result[1]
f,s=tf.math.sigmoid(f).cpu().numpy(),tf.math.sigmoid(s).cpu().numpy()

print("faction for tweet: ", f[0], f[1])
print("sentiment for tweet: ", s[0], s[1])

1/1 [==============================] - 0s 71ms/step


Instructions for updating:
Use tf.identity instead.


faction for tweet:  [0.98713243] [0.07715884]
sentiment for tweet:  [0.9762982] [0.08260459]
